In [1]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertTokenizer
import pandas as pd
import numpy as np
from sklearn.utils import resample

In [2]:
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


Torch version: 2.5.1+cu121
CUDA available: True
Device name: NVIDIA GeForce RTX 3050 Laptop GPU


### `Завантаження датасету`

In [ ]:
data = pd.read_csv("../data/spam.csv", encoding='latin-1')

data = data[['v1', 'v2']]
data.columns = ['label', 'message']

data['label'] = data['label'].map({'ham': 0, 'spam': 1})

print("Файл завантажено!")
print("Розмір датасету:", data.shape)
print(data.sample(5))


Файл завантажено!
Розмір датасету: (5572, 2)
      label                                            message
1011      0        I just got home babe, are you still awake ?
4833      0                      I hope your pee burns tonite.
4577      1  Urgent! call 09066350750 from your landline. Y...
2570      0                   From 5 to 2 only my work timing.
2354      0               R we going with the  &lt;#&gt;  bus?


pandas.core.frame.DataFrame

In [4]:
for a in data['label'].unique():
    print(f"Клас {a} має {data[data['label'] == a].shape[0]} прикладів") 

Клас 0 має 4825 прикладів
Клас 1 має 747 прикладів


### `Токенізація`

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(texts, tokenizer, max_len=64):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_len, return_tensors="pt")

### `Підготовка датасету та балансування класів`

In [6]:
majority = data[data['label'] == 0]  # ham
minority = data[data['label'] == 1]  # spam

print("До балансування:")
print("ham:", len(majority), "| spam:", len(minority))

# Oversampling меншого класу (spam)
minority_upsampled = resample(
    minority,
    replace=True,                # з повторенням
    n_samples=len(majority),     # до кількості ham
    random_state=42
)

balanced_data = pd.concat([majority, minority_upsampled]).sample(frac=1, random_state=42).reset_index(drop=True)

balanced_data.to_csv('../data/balanced_data.csv', index=False)


print("\nПісля балансування:")
print(balanced_data['label'].value_counts())

train_texts, test_texts, train_labels, test_labels = train_test_split(
    balanced_data['message'], balanced_data['label'], test_size=0.2, random_state=42, stratify=balanced_data['label']
)

train_encodings = tokenize_data(train_texts, tokenizer)
test_encodings = tokenize_data(test_texts, tokenizer)

class SpamDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SpamDataset(train_encodings, train_labels)
test_dataset = SpamDataset(test_encodings, test_labels)


До балансування:
ham: 4825 | spam: 747

Після балансування:
label
0    4825
1    4825
Name: count, dtype: int64


### `Архітектура моделі`
#### `PositionalEncoding, Embedding, TransformerEncoderLayer(2 шари), Linear`

In [7]:
# Позиційне кодування
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

class SimpleTransformerClassifier(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers=2, num_classes=2, dim_feedforward=256):

        super(SimpleTransformerClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            batch_first=False  # permute(1,0,2)
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)
    
    def forward(self, input_ids):
        x = self.embedding(input_ids)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)  # (seq_len, batch, d_model)
        x = self.transformer(x)
        x = x.mean(dim=0)       
        out = self.fc(x)
        return out


### `Тренування моделі та тюнінг гіперпараметрів`

In [ ]:
from itertools import product

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

d_models = [128, 256, 512]  
nheads = [2, 4, 8]           
num_epochs = 3

best_f1 = 0
best_params = None

for d_m, nh in product(d_models, nheads):
    print(f"\n-> Навчання моделі з параметрами d_model={d_m}, nhead={nh}")
    model = SimpleTransformerClassifier(
        vocab_size=tokenizer.vocab_size,
        d_model=d_m,
        nhead=nh,
        num_layers=2
    ).to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"  Epoch {epoch+1}/{num_epochs}, avg loss = {total_loss / len(train_loader):.4f}")


-> Навчання моделі з параметрами d_model=128, nhead=2


C:\Users\gnatu\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


  Epoch 1/3, avg loss = 0.3016
  Epoch 2/3, avg loss = 0.1069


### `Оцінка та збереження моделі`

In [ ]:
model.eval()
preds, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids)
        preds += torch.argmax(outputs, dim=1).cpu().tolist()
        true_labels += labels.cpu().tolist()

acc = accuracy_score(true_labels, preds)
prec = precision_score(true_labels, preds)
rec = recall_score(true_labels, preds)
f1 = f1_score(true_labels, preds)

print(f"  🔹 Acc: {acc:.5f} | Prec: {prec:.5f} | Rec: {rec:.5f} | F1: {f1:.5f}")

if f1 > best_f1:
    best_f1 = f1
    best_params = (d_m, nh)
    torch.save(model.state_dict(), "best_simple_transformer.pt")
    print("  ✅ Нові найкращі ваги збережено!")

# цей print уже після всіх перевірок
print(f"\n🏆 Найкраща комбінація: d_model={best_params[0]}, nhead={best_params[1]} з F1={best_f1:.6f}")

  🔹 Acc: 0.99689 | Prec: 0.99382 | Rec: 1.00000 | F1: 0.99690
  ✅ Нові найкращі ваги збережено!

🏆 Найкраща комбінація: d_model=512, nhead=8 з F1=0.996901
